In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import plotly
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)
import seaborn as sns
import matplotlib as mpl
import re
import pandas as pd
import json
import os
import sys
import glob

In [3]:
import pandas as pd
import json
import os
import csv 
from tqdm import tqdm

data = {}
name = {}
file_path = ''
paths = glob.glob(f'_trial_*{file_path}*')
#paths = glob.glob(f'_trial_*')
for path in tqdm(paths):
    agent = path.split("_")[-6]
    exploration_strategy = path.split("_")[-5]
    grid = path.split("_")[-4]
    ghost = "_".join(path.split("_")[-3:-1])
    noise = "".join(path.split("_")[-1])
    pkl_files = glob.glob(os.path.join(path, 'saved_agent*.pkl'))
    for file in pkl_files:       
        if "learnability" in path:
            otherghost = ghost
            othernoise = noise
        else:
            repeat_grid = file.split("_")[3]
            path_file = f"_{repeat_grid}" + re.findall(r'-train.*?_end', file)[0]
            path_file = path_file.replace("'","\"").replace(" ", "").replace("-train","").replace("-test","").replace("_end", "")
            otherghost = "_".join(path_file.split("_")[2:4])
            othernoise = path_file.split("_")[4]
            
        if agent not in data:
            data[agent] = {}
            name[agent] = {}
        if exploration_strategy not in data[agent]:
            data[agent][exploration_strategy] = {}
            name[agent][exploration_strategy] = {}
        if grid not in data[agent][exploration_strategy]:
            data[agent][exploration_strategy][grid] = {}
            name[agent][exploration_strategy][grid] = {}
        if ghost not in data[agent][exploration_strategy][grid]:
            data[agent][exploration_strategy][grid][ghost] = {}
            name[agent][exploration_strategy][grid][ghost] = {}
        if noise not in data[agent][exploration_strategy][grid][ghost]:
            data[agent][exploration_strategy][grid][ghost][noise] = {}
            name[agent][exploration_strategy][grid][ghost][noise] = {}
        if otherghost not in data[agent][exploration_strategy][grid][ghost][noise]:
            data[agent][exploration_strategy][grid][ghost][noise][otherghost] = {}
            name[agent][exploration_strategy][grid][ghost][noise][otherghost] = {}
        if othernoise not in data[agent][exploration_strategy][grid][ghost][noise][otherghost]:
            name[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise] = []
            data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise] = {}
            data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise]["tables"] = []
    
        values = []
        with open(file, newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter='\n', quotechar='|')
            for row in reader:
                values.append(float(row[0]))
        data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise]["values"] = values
    
        json_files = glob.glob(os.path.join(path, '*epoch.json')) 
        for file in json_files:
            with open(file) as f:
                train_epoch = file.split("-")[-1].replace(rf"train0_","").replace(".json","")
                data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise]["tables"].append(json.load(f))
                name[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise].append(f'{re.sub("./_trial_", "", path)}_{train_epoch}')

  0%|                                                                                                                                                         | 0/156 [00:00<?, ?it/s]

KeyboardInterrupt



In [4]:
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import os
import imageio

def generate_occupancy(folder,subfolder,agent, exploration_strategy, somegrid, someghost, somenoise, someotherghost, someothernoise, sortex_idx):
    
    states_distribution = {}
    if agent not in states_distribution:
        states_distribution[agent] = {}
    if exploration_strategy not in states_distribution[agent]:
        states_distribution[agent][exploration_strategy] = {}
    if somegrid not in states_distribution[agent][exploration_strategy]:
        states_distribution[agent][exploration_strategy][somegrid] = {}
    if someghost not in states_distribution[agent][exploration_strategy][somegrid]:
        states_distribution[agent][exploration_strategy][somegrid][someghost] = {}
    if somenoise not in states_distribution[agent][exploration_strategy][somegrid][someghost]:
        states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise] = {}
    if someotherghost not in states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise]:
        states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost] = {}
    if someothernoise not in states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost]:
        states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost][someothernoise] = {}
        
    idxs = sorted(range(len(name[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise])), key=lambda x: int(name[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise][x].split('_')[-2]))
    evolution_game = np.asarray(data[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise]["tables"])[idxs]
    idxs_o = sorted(range(len(name[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise])), key=lambda x: int(name[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise][x].split('_')[-2]))
    evolution_game_other = np.asarray(data[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise]["tables"])[idxs_o]
    action_pd_no_noise = pd.DataFrame(evolution_game[-1]).sort_index()
    action_pd_noise = pd.DataFrame(evolution_game_other[-1]).sort_index()
    all_columns = pd.merge(action_pd_no_noise.fillna(np.nan).astype(float), action_pd_noise.fillna(np.nan).astype(float), how="outer").columns
    action_pd_no_noise_complete = pd.concat([action_pd_no_noise,pd.DataFrame(columns=list(set(all_columns) - set(action_pd_no_noise.columns)))]).fillna(np.nan).sort_index()[all_columns]
    action_pd_noise_complete = pd.concat([action_pd_noise,pd.DataFrame(columns=list(set(all_columns) - set(action_pd_noise.columns)))]).fillna(np.nan).sort_index()[all_columns]
    common_nan_mask = np.isnan(action_pd_no_noise_complete.values) & np.isnan(action_pd_noise_complete.values)
    column_mask = np.all(common_nan_mask, axis=0)
    masked_data_terminal = np.ma.masked_array(common_nan_mask, mask=np.repeat(column_mask.reshape(1,-1), 4, axis=0))
    
    #for sortex_idx in range(len(data[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise]["values"])):
    #fig, ax = plt.subplots(1,1, figsize=(27,27))
    action_pd_no_noise = pd.DataFrame(evolution_game[sortex_idx]).sort_index()
    action_pd_noise = pd.DataFrame(evolution_game_other[sortex_idx]).sort_index()

    action_pd_no_noise_complete = pd.concat([action_pd_no_noise,pd.DataFrame(columns=list(set(all_columns) - set(action_pd_no_noise.columns)))]).fillna(np.nan).sort_index()[all_columns]
    action_pd_noise_complete = pd.concat([action_pd_noise,pd.DataFrame(columns=list(set(all_columns) - set(action_pd_noise.columns)))]).fillna(np.nan).sort_index()[all_columns]
    common_nan_mask = np.isnan(action_pd_no_noise_complete.values) & np.isnan(action_pd_noise_complete.values)
    action_pd_noise_nan_mask = np.isnan(action_pd_noise_complete.values) & ~np.isnan(action_pd_no_noise_complete.values)
    action_pd_no_noise_nan_mask = np.isnan(action_pd_no_noise_complete.values) & ~np.isnan(action_pd_noise_complete.values)

    merge = pd.merge(action_pd_no_noise_complete, action_pd_noise_complete, how='outer')[all_columns]
    a = abs(merge.iloc[:4,:].to_numpy() - merge.iloc[4:,:].to_numpy())
    
    if np.any(action_pd_noise_nan_mask == True):
        reds = np.sum(action_pd_noise_nan_mask)
    else:
        reds = 0
    if np.any(action_pd_no_noise_nan_mask == True):
        blues = np.sum(action_pd_no_noise_nan_mask)
    else:
        blues = 0
    greens = np.sum(masked_data_terminal.mask)
    states = np.sum(~common_nan_mask) - reds - blues
    G = data[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost][someothernoise]["values"][sortex_idx]
    L = data[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise]["values"][sortex_idx]
    
    return [reds, blues, states]
    

In [5]:
def nanargmax_with_default(series):
    return 0 if series.isna().all() else np.nanargmax(series)

def generate_inner_explored_states(folder, subfolder, agent, exploration_strategy, somegrid, someghost, somenoise, someotherghost, someothernoise, sortex_idx):
    
    states_distribution = {}
    if agent not in states_distribution:
        states_distribution[agent] = {}
    if exploration_strategy not in states_distribution[agent]:
        states_distribution[agent][exploration_strategy] = {}
    if somegrid not in states_distribution[agent][exploration_strategy]:
        states_distribution[agent][exploration_strategy][somegrid] = {}
    if someghost not in states_distribution[agent][exploration_strategy][somegrid]:
        states_distribution[agent][exploration_strategy][somegrid][someghost] = {}
    if somenoise not in states_distribution[agent][exploration_strategy][somegrid][someghost]:
        states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise] = {}
    if someotherghost not in states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise]:
        states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost] = {}
    if someothernoise not in states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost]:
        states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost][someothernoise] = {}
        
    idxs = sorted(range(len(name[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise])), key=lambda x: int(name[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise][x].split('_')[-2]))
    evolution_game = np.asarray(data[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise]["tables"])[idxs]
    idxs_o = sorted(range(len(name[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise])), key=lambda x: int(name[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise][x].split('_')[-2]))
    evolution_game_other = np.asarray(data[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise]["tables"])[idxs_o]

    action_pd_no_noise = pd.DataFrame(evolution_game[-1]).sort_index()
    action_pd_noise = pd.DataFrame(evolution_game_other[-1]).sort_index()
    all_columns = action_pd_noise.columns.intersection(action_pd_no_noise.columns)
    merge_common = pd.concat([action_pd_noise[all_columns], action_pd_no_noise[all_columns]], axis=0, keys=['action_pd_no_noise', 'action_pd_noise'])[all_columns]
    merge_common.index = merge_common.index.droplevel()
    final_column = np.argsort(merge_common.iloc[:4,:].apply(nanargmax_with_default).to_numpy() == merge_common.iloc[4:,:].apply(nanargmax_with_default).to_numpy())[::-1]
    common_nan_mask = np.isnan(merge_common.iloc[:4,:].values) & np.isnan(merge_common.iloc[4:,:].values)
    column_mask = np.all(common_nan_mask, axis=0)[final_column]
    masked_data_terminal = np.ma.masked_array(common_nan_mask, mask=np.repeat(column_mask.reshape(1,-1), 4, axis=0))
    all_columns = all_columns[final_column]
    directory_path = f"{folder}/{subfolder}"

    #for sortex_idx in range(len(data[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise]["values"])):

    action_pd_no_noise = pd.DataFrame(evolution_game[sortex_idx]).sort_index()
    action_pd_noise = pd.DataFrame(evolution_game_other[sortex_idx]).sort_index()

    common_columns = action_pd_no_noise.columns.intersection(action_pd_noise.columns)
    action_pd_no_noise_common = pd.concat([action_pd_no_noise[common_columns],pd.DataFrame(columns=list(set(all_columns) - set(common_columns)))]).fillna(np.nan).sort_index()[all_columns]
    action_pd_noise_common = pd.concat([action_pd_noise[common_columns],pd.DataFrame(columns=list(set(all_columns) - set(common_columns)))]).fillna(np.nan).sort_index()[all_columns]

    merge_common = pd.concat([action_pd_no_noise_common, action_pd_noise_common], axis=0, keys=['action_pd_no_noise', 'action_pd_noise'])[all_columns]
    a = pd.DataFrame(merge_common.iloc[:4,:].apply(nanargmax_with_default).to_numpy() == merge_common.iloc[4:,:].apply(nanargmax_with_default).to_numpy(), index=all_columns).values.astype(float)
    states = a.shape[0]
    common = states - np.sum(a)
    perc_common = common/states
    G = data[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost][someothernoise]["values"][sortex_idx]
    L = data[agent][exploration_strategy][somegrid][someotherghost][someothernoise][someotherghost][someothernoise]["values"][sortex_idx]
    states_distribution[agent][exploration_strategy][somegrid][someghost][somenoise][someotherghost][someothernoise][sortex_idx] = [perc_common, L, G]     
    return [perc_common, L, G]

In [6]:
import pickle

# NON SEMANTIC NOISE
somegrid_l = ['v2','v3', 'v4']
someagent_l = ['BoltzmannAgent','SarsaAgent']
someexploration_strategy_l = ['Boltzmann','Egreedy']
#training env
someghost_l = ['RandomGhost_{"index":1,"prob":{}}','DirectionalGhost_{"index":1,"prob":0.3}','DirectionalGhost_{"index":1,"prob":0.6}'] 
somenoise_l = ['{"mean":0,"std":0}','{"mean":0,"std":0}','{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhost_{"index":1,"prob":{}}','DirectionalGhost_{"index":1,"prob":0.3}','DirectionalGhost_{"index":1,"prob":0.6}']
someothernoise_l = ['{"mean":0,"std":0.1}','{"mean":0,"std":0.1}','{"mean":0,"std":0.1}']

In [7]:
# SEMANTIC NOISE
somegrid_l = ['v2','v3', 'v4']
someagent_l = ['SarsaAgent','BoltzmannAgent']
someexploration_strategy_l = ['Boltzmann','Egreedy']
#training env
someghost_l = ['RandomGhost_{"index":1,"prob":{}}']
somenoise_l = ['{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhostTeleportingNearWalls_{"index":1,"prob":{}}']
someothernoise_l =  ['{"mean":0,"std":0}']

In [8]:
data[agent][exploration_strategy][somegrid][someghost][somenoise].keys()

NameError: name 'somegrid' is not defined

In [12]:
for someagent in someagent_l:
    if someagent not in name:
        continue
    for someexploration_strategy in someexploration_strategy_l:
        if someexploration_strategy not in name[someagent]:
            continue
        for somegrid in somegrid_l:
            if somegrid not in name[someagent][someexploration_strategy]:
                continue
            for someghost, somenoise, someotherghost, someothernoise in zip(someghost_l, somenoise_l, someotherghost_l, someothernoise_l):
                if someghost not in name[someagent][someexploration_strategy][somegrid]:
                    continue
                if somenoise not in name[someagent][someexploration_strategy][somegrid][someghost]:
                    continue
                input_string = np.asarray(name[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise])[0]
                folder = re.sub(r"_training_agent_\d+_epoch", "", input_string)
                subfolder = f"results_{somegrid}_{someghost}_{somenoise}_{someotherghost}_{someothernoise}"
                output_name = re.sub("results_","",subfolder)
                pickle_file_path = f"{folder}/{subfolder}.pkl"
                values = []
                for sortex_idx in range(len(data[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise]["values"])):
                    a = generate_inner_explored_states(folder,subfolder,someagent, someexploration_strategy, somegrid, someghost, somenoise, someotherghost, someothernoise, sortex_idx)
                    b = generate_occupancy(folder,subfolder,agent, exploration_strategy,somegrid, someghost, somenoise, someotherghost, someothernoise, sortex_idx)
                    values.append(b + a)
                    print(b + a)


[503, 417, 186, 0.13941018766756033, -303.6, -507.1000061035156]
[396, 736, 293, 0.2225201072386059, -505.8, -407.1000061035156]
[417, 819, 477, 0.3297587131367292, -308.1, -505.79998779296875]
[354, 923, 540, 0.36193029490616624, -209.8, -311.29998779296875]
[361, 937, 605, 0.4075067024128686, -205.5, -506.79998779296875]
[335, 995, 631, 0.4343163538873995, -517.1, -317.20001220703125]
[319, 1002, 686, 0.4959785522788204, -205.9, -512.4000244140625]
[295, 1021, 710, 0.4959785522788204, -201.8, -417.1000061035156]
[270, 1020, 753, 0.5201072386058981, -303.3, 92.0]
[265, 1024, 758, 0.5415549597855228, -303.7, -102.19999694824219]
[257, 1032, 781, 0.5361930294906166, -208.9, -111.80000305175781]
[254, 1050, 784, 0.5361930294906166, -217.1, -200.89999389648438]
[253, 1044, 797, 0.5683646112600537, -305.5, 92.9000015258789]
[251, 1061, 799, 0.579088471849866, -202.7, -4.300000190734863]
[254, 1070, 805, 0.5951742627345844, -7.6, 95.19999694824219]
[252, 1071, 807, 0.6005361930294906, -108.


KeyboardInterrupt



In [2]:
import pandas as pd
import json
import os
import csv 
from tqdm import tqdm

data = {}
name = {}
file_path = ''
paths = glob.glob(f'_trial_*{file_path}*')
#paths = glob.glob(f'_trial_*')
for path in tqdm(paths):
    agent = path.split("_")[-6]
    exploration_strategy = path.split("_")[-5]
    grid = path.split("_")[-4]
    ghost = "_".join(path.split("_")[-3:-1])
    noise = "".join(path.split("_")[-1])
    pkl_files = glob.glob(os.path.join(path, 'saved_agent*.pkl'))
    for file in pkl_files:       
        if "learnability" in path:
            otherghost = ghost
            othernoise = noise
        else:
            repeat_grid = file.split("_")[3]
            path_file = f"_{repeat_grid}" + re.findall(r'-train.*?_end', file)[0]
            path_file = path_file.replace("'","\"").replace(" ", "").replace("-train","").replace("-test","").replace("_end", "")
            otherghost = "_".join(path_file.split("_")[2:4])
            othernoise = path_file.split("_")[4]
            
        if agent not in data:
            data[agent] = {}
            name[agent] = {}
        if exploration_strategy not in data[agent]:
            data[agent][exploration_strategy] = {}
            name[agent][exploration_strategy] = {}
        if grid not in data[agent][exploration_strategy]:
            data[agent][exploration_strategy][grid] = {}
            name[agent][exploration_strategy][grid] = {}
        if ghost not in data[agent][exploration_strategy][grid]:
            data[agent][exploration_strategy][grid][ghost] = {}
            name[agent][exploration_strategy][grid][ghost] = {}
        if noise not in data[agent][exploration_strategy][grid][ghost]:
            data[agent][exploration_strategy][grid][ghost][noise] = {}
            name[agent][exploration_strategy][grid][ghost][noise] = {}
        if otherghost not in data[agent][exploration_strategy][grid][ghost][noise]:
            data[agent][exploration_strategy][grid][ghost][noise][otherghost] = {}
            name[agent][exploration_strategy][grid][ghost][noise][otherghost] = {}
        if othernoise not in data[agent][exploration_strategy][grid][ghost][noise][otherghost]:
            name[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise] = []
            data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise] = {}
            data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise]["tables"] = []
    
        values = []
        with open(file, newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter='\n', quotechar='|')
            for row in reader:
                values.append(float(row[0]))
        data[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise]["values"] = values
    
        json_files = glob.glob(os.path.join(path, '*epoch.json')) 
        for file in json_files:
            train_epoch = file.split("-")[-1].replace(rf"train0_","").replace(".json","")
            name[agent][exploration_strategy][grid][ghost][noise][otherghost][othernoise].append(f'{re.sub("./_trial_", "", path)}_{train_epoch}')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [00:01<00:00, 85.99it/s]


In [3]:
name['BoltzmannAgent']['Boltzmann']['v2'].keys()

dict_keys(['DirectionalGhost_{"index":1,"prob":0.6}', 'RandomGhost_{"index":1,"prob":{}}', 'RandomGhostTeleportingNearWalls_{"index":1,"prob":{}}', 'DirectionalGhost_{"index":1,"prob":0.3}'])

In [4]:
import numpy as np
import scipy

def test_significance(x,y):
    return scipy.stats.wilcoxon(x,y)

In [5]:
import pickle

# NON SEMANTIC NOISE
somegrid_l = ['v2','v3','v4']
normalization_factors_l = [982, 466, 626]
someagent_l = ['BoltzmannAgent','SarsaAgent']
someexploration_strategy_l = ['Boltzmann','Egreedy']
#training env
someghost_l = ['RandomGhost_{"index":1,"prob":{}}','DirectionalGhost_{"index":1,"prob":0.3}','DirectionalGhost_{"index":1,"prob":0.6}','RandomGhost_{"index":1,"prob":{}}'] 
somenoise_l = ['{"mean":0,"std":0}','{"mean":0,"std":0}','{"mean":0,"std":0}','{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhost_{"index":1,"prob":{}}','DirectionalGhost_{"index":1,"prob":0.3}','DirectionalGhost_{"index":1,"prob":0.6}','RandomGhostTeleportingNearWalls_{"index":1,"prob":{}}']
someothernoise_l = ['{"mean":0,"std":0.1}','{"mean":0,"std":0.1}','{"mean":0,"std":0.1}','{"mean":0,"std":0}']

In [6]:
import pickle

# SEMANTIC NOISE
somegrid_l = ['v2','v3','v4']
normalization_factors_l = [982, 466, 626]
someagent_l = ['BoltzmannAgent','SarsaAgent']
someexploration_strategy_l = ['Boltzmann','Egreedy']
#training env
someghost_l = ['RandomGhost_{"index":1,"prob":{}}'] 
somenoise_l = ['{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhostTeleportingNearWalls_{"index":1,"prob":{}}']
someothernoise_l = ['{"mean":0,"std":0}']

In [7]:
import pickle

# NON SEMANTIC NOISE
somegrid_l = ['v2','v3','v4']
normalization_factors_l = [982, 466, 626]

someagent_l = ['BoltzmannAgent','SarsaAgent']
someexploration_strategy_l = ['Boltzmann','Egreedy']
#training env
someghost_l = ['RandomGhost_{"index":1,"prob":{}}','DirectionalGhost_{"index":1,"prob":0.3}','DirectionalGhost_{"index":1,"prob":0.6}'] 
somenoise_l = ['{"mean":0,"std":0}','{"mean":0,"std":0}','{"mean":0,"std":0}']
#testing env
someotherghost_l = ['RandomGhost_{"index":1,"prob":{}}','DirectionalGhost_{"index":1,"prob":0.3}','DirectionalGhost_{"index":1,"prob":0.6}']
someothernoise_l = ['{"mean":0,"std":0.1}','{"mean":0,"std":0.1}','{"mean":0,"std":0.1}']

In [17]:
contents_lg = {}
contents_lg['blue'] = []
contents_lg['red'] = []
contents_lg['states'] = []
contents_lg['perc_common'] = []
gap_lg = []

for someagent in someagent_l:
    if someagent not in name:
        continue
    for someexploration_strategy in someexploration_strategy_l:
        if someexploration_strategy not in name[someagent]:
            continue
        for somegrid, norm in zip(somegrid_l, normalization_factors_l):
            if somegrid not in name[someagent][someexploration_strategy]:
                continue
            for someghost, somenoise, someotherghost, someothernoise in zip(someghost_l, somenoise_l, someotherghost_l, someothernoise_l):
                if someghost not in name[someagent][someexploration_strategy][somegrid]:
                    continue
                if somenoise not in name[someagent][someexploration_strategy][somegrid][someghost]:
                    continue
                input_string = np.asarray(name[agent][exploration_strategy][somegrid][someghost][somenoise][someghost][somenoise])[0]
                folder = re.sub(r"_training_agent_\d+_epoch", "", input_string)
                subfolder = f"results_{somegrid}_{someghost}_{somenoise}_{someotherghost}_{someothernoise}"
                output_name = re.sub("results_","",subfolder)
                file = f"{folder}/{subfolder}.pkl"
                with open(file, 'rb') as file:
                    values = pickle.load(file)
                for value in values[-10:]:
                    contents_lg['red'].append(value[0]/(norm*4))
                    contents_lg['blue'].append(value[1]/(norm*4))
                    contents_lg['states'].append(value[2]/(norm*4))
                    contents_lg['perc_common'].append(value[3])
                    gap_lg.append((value[5] - value[4])/np.abs(value[5]))
                

## The sum or Reds and Blues is a good predictor of G>L

In [18]:
total_blues_reds = np.array(contents_lg['blue']) + np.array(contents_lg['red'])
scipy.stats.ttest_ind(total_blues_reds[np.where(np.array(gap_lg)>0)], total_blues_reds[np.where(np.array(gap_lg)<0)])

TtestResult(statistic=-9.48224716568476, pvalue=3.548919764935312e-19, df=358.0)

In [19]:
print( "MEAN:", np.mean(total_blues_reds[np.where(np.array(gap_lg)>0)]),"STD:", scipy.stats.sem(total_blues_reds[np.where(np.array(gap_lg)>0)])) 
print( "MEAN:", np.mean(total_blues_reds[np.where(np.array(gap_lg)<0)]),"STD:",  scipy.stats.sem(total_blues_reds[np.where(np.array(gap_lg)<0)]))

MEAN: 0.08367238081395567 STD: 0.011268688365028584
MEAN: 0.27100433401969 STD: 0.01712763227778641


In [54]:
# Pearson's correlation test
pearson_corr, pearson_p_value = scipy.stats.pearsonr(total_blues_reds, gap_lg)
print(f"Pearson's correlation coefficient: {pearson_corr}")
print(f"Pearson's p-value: {pearson_p_value}")

# Spearman's correlation test
spearman_corr, spearman_p_value = scipy.stats.spearmanr(total_blues_reds, gap_lg)
print(f"Spearman's correlation coefficient: {spearman_corr}")
print(f"Spearman's p-value: {spearman_p_value}")

Pearson's correlation coefficient: 0.04692360014216373
Pearson's p-value: 0.3746992025571309
Spearman's correlation coefficient: -0.4222662123833527
Spearman's p-value: 5.328027055321341e-17


## The % of states that have same preferred action is a good predictor of G>L

In [29]:
perc_common = np.asarray(contents_lg['perc_common'])
scipy.stats.ttest_ind(perc_common[np.where(np.array(gap_lg)>0)], perc_common[np.where(np.array(gap_lg)<0)])

TtestResult(statistic=-8.7520129662423, pvalue=8.386318852248445e-17, df=358.0)

In [30]:
# Pearson's correlation test
pearson_corr, pearson_p_value = scipy.stats.pearsonr(contents_lg['perc_common'], gap_lg)
print(f"Pearson's correlation coefficient: {pearson_corr}")
print(f"Pearson's p-value: {pearson_p_value}")

# Spearman's correlation test
spearman_corr, spearman_p_value = scipy.stats.spearmanr(contents_lg['perc_common'], gap_lg)
print(f"Spearman's correlation coefficient: {spearman_corr}")
print(f"Spearman's p-value: {spearman_p_value}")

Pearson's correlation coefficient: 0.06367745198930215
Pearson's p-value: 0.22811980015776082
Spearman's correlation coefficient: -0.25009401155405653
Spearman's p-value: 1.5454066617900714e-06
